In [1]:
import os
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/sujin/PycharmProjects/OpenRLHF")

from openrlhf.utils import blending_datasets, get_strategy, get_tokenizer
from datasets import load_dataset, concatenate_datasets, interleave_datasets
from openrlhf.datasets import SFTDataset, PromptDataset
from easydict import EasyDict

/root/miniconda3/envs/openrlhf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pretrain = "/sujin/Models/Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = get_tokenizer(pretrain, None, "right", None, use_fast=True)

In [27]:
dataset = "Open-Orca/OpenOrca"

strategy = {
    "args": {
        "input_key": "question",
        "output_key": "response",
        "input_template": "$'User: {}\nAssistant: '",
        "apply_chat_template": False,
        "tokenizer_chat_template": None
    }
}
strategy = EasyDict(strategy)

train_data = load_dataset(dataset)
train_data = concatenate_datasets([train_data["train"]])
train_data = train_data.select(range(1000))

train_dataset = SFTDataset(
    train_data,
    tokenizer,
    4096,
    strategy,
    pretrain_mode=False,
    input_template=strategy["args"]["input_template"],
    multiturn=False,
)

Filter: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 77417.11 examples/s]


In [41]:
train_dataset.__getitem__(100)

(tensor([[     3,      6,   1474,     25,  29901,    279,  25975,   1447,   1654,
           19334,    279,    348,   5082,    323,   1730,    432,    311,    387,
           26291,     13,   2806,  14063,   4113,    911,    419,    830,   3364,
              11,    358,   3381,     25,    330,  11908,     11,    902,     11,
             393,   1785,   3630,  18759,    438,    458,   3693,   7748,   4319,
            1151,      6,   7748,    304,    279,   4531,    701,   1128,    264,
            3873,   5754,      1,   3080,    358,  11105,    279,   8046,    911,
           25685,  73695,     13,    576,   4531,   1558,    264,   1661,   2618,
             315,  44196,    279,  37829,    315,   1493,   9867,  31438,    323,
           77369,    279,  24207,  50186,    911,   1105,     13,   1597,  51989,
           10620,   3444,   2863,     11,  61300,     11,   1128,    264,  13143,
              11,    323,   1602,  39561,    438,    458,   7748,   5220,    320,
              40

# Prompt dataset

In [9]:
dataset = "OpenRLHF/prompt-collection-v0.1"

strategy = {
    "args": {
        "input_key": "context_messages",
        "input_template": None,
        "apply_chat_template": True,
    },
    "is_rank_0": lambda: False,
}
strategy = EasyDict(strategy)

train_data = load_dataset(dataset)
train_data = concatenate_datasets([train_data["train"]])
train_data = train_data.select(range(1000))

In [24]:
# Create train dataset
prompts_dataset = PromptDataset(train_data, tokenizer, strategy, input_template=strategy.args.input_template)
print(prompts_dataset.__getitem__(0))

('default', '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nWhat are the three most important things to consider when deciding what technology to use to build an assist device to help an elderly person with basic needs?<|im_end|>\n<|im_start|>assistant\n', '')


In [31]:
from openrlhf.models.model import get_llm_for_sequence_regression


get_llm_for_sequence_regression("/sujin/Models/Qwen/Qwen2.5-0.5B-Instruct", "reward")

INFO 05-07 10:04:14 model.py:73] set value_head_prefix to `score`


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.
Some weights of RewardModel were not initialized from the model checkpoint at /sujin/Models/Qwen/Qwen2.5-0.5B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RewardModel(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbedding

In [39]:
import torch
 
# 创建一个大小为 (2, 4, 3) 的输入张量
input_tensor = torch.tensor([
    [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]],
    [[13, 14, 15], [16, 17, 18], [19, 20, 21], [22, 23, 24]]
])
 
# 创建一个大小为 (2, 2) 的索引张量
indices = torch.tensor([[1, 3], [0, 2]])
 
# 从每个批次的张量中收集指定位置的元素
gathered_values = torch.gather(input_tensor, dim=1, index=indices.unsqueeze(-1).repeat(1, 1, input_tensor.shape[-1]))

print(indices.unsqueeze(-1).repeat(1, 1, input_tensor.shape[-1]))
print(gathered_values)

tensor([[[1, 1, 1],
         [3, 3, 3]],

        [[0, 0, 0],
         [2, 2, 2]]])
tensor([[[ 4,  5,  6],
         [10, 11, 12]],

        [[13, 14, 15],
         [19, 20, 21]]])
